<br>
<br>

### Proyecto: Análisis de emociones

##### ![1) Definición del objetivo:](https://github.com/chetincho/ds_Prediccion_de_enfermedades/blob/main/img/Definici%C3%B3n%20del%20objetivo.jpg?raw=true)

.. xx ..<br><br>
Se busca obtener un modelo computacional que permita la predicción de enfermedades, dicho modelo se convertirá en un soporte para la toma de decisiones médicas cuya aplicación estará enfocada principalmente en estudiantes de medicina recién graduados que estén atravesando por su primer año de residencia.

##### ![2) Contexto comercial:](https://github.com/chetincho/ds_Prediccion_de_enfermedades/blob/main/img/Contexto%20comercial.jpg?raw=true)

.. xx ..<br><br>
El resultado final del proyecto tiene por objetivo:<br>
1- Minimizar inconvenientes legales causados por negligencias médicas proveniente de la inexperiencia de los recién graduados.<br>
2- Proveer a los internos de un soporte a sus inferencias médicas ante una emergencia o la ausencia de un titular.<br>

##### ![3) Problema comercial:](https://github.com/chetincho/ds_Prediccion_de_enfermedades/blob/main/img/Problema%20comercial.jpg?raw=true)

.. xx ..<br><br>
¿Es posible desarrollar un modelo que, en función de ciertos inputs, nos permita predecir una enfermedad?

##### ![Librerías utilizadas:](https://github.com/chetincho/ds_Prediccion_de_enfermedades/blob/main/img/Librerias%20utilizadas.jpg?raw=true)

In [27]:
import pandas as pd


# Importamos el módulo re de Python el cual nos permite utilizar expresiones regulares.
import re


# Importamos la librería nltk, una de las bibliotecas más comunes y robustas en Python
# utilizadas para la eliminación de stopwords
import nltk
from nltk.corpus import stopwords # Importamos las stopwords

# Librerias de Lematización
import spacy # Importamos spaCy

##### ![4) Data Acquisition:](https://github.com/chetincho/ds_Prediccion_de_enfermedades/blob/main/img/Data%20Acquisition.jpg?raw=true)

.. xx ..<br><br>
El dataset utilizado es el resultado de la recopilación de datos provenientes de las historias clínicas de pacientes que ya fueron diagnosticados por esta institución y cuyos casos fueron analizados por la junta médica de este hospital.

In [28]:
# Origen de los datos
url_dataset="https://raw.githubusercontent.com/chetincho/Analisis_de_Sentimientos/refs/heads/main/dataset/dataset.txt"

# Carga del dataframe
df = pd.read_csv(url_dataset, sep='|')

# Seteamos el índice del dataframe para que comience en 1
df.set_index(pd.Index(range(1, len(df) + 1)), inplace=True)

# Seteamos pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

##### ![5) Exploratory Data Analysis (EDA):](https://github.com/chetincho/ds_Prediccion_de_enfermedades/blob/main/img/Exploratory%20Data%20Analysis.jpg?raw=true)

🗒️ ¿Cuál es la cantidad de filas y columnas que componen el dataframe?

In [29]:
filas, columnas = df.shape
print(f"✅ Total de Filas = {filas}")
print(f"✅ Total de Columnas = {columnas}")

✅ Total de Filas = 3316
✅ Total de Columnas = 19


🗒️ ¿Cuáles son las columnas o atributos que lo componen?

In [30]:
# Utilizo el método .tolist() para convertir las columnas en una lista
print(f"Este dataframe está compuesto por las siguientes columnas:")
columnas = df.columns.tolist()
for columna in columnas:
    print(f"📁 {columna}")

Este dataframe está compuesto por las siguientes columnas:
📁 Usuario X
📁 ID
📁 Text
📁 Language
📁 Type
📁 Author Name
📁 Author Username
📁 View Count
📁 Bookmark Count
📁 Favorite Count
📁 Retweet Count
📁 Reply Count
📁 Quote Count
📁 Created At
📁 Source
📁 Hashtags
📁 URLs
📁 Media Type
📁 Media URLs


🗒️ ¿Hay valores nulos?

In [31]:
print(f"ℹ️ Total de valores nulos detectados: {sum(df.isnull().sum())} valores")

ℹ️ Total de valores nulos detectados: 9753 valores


🗒️ ¿Cuáles son los atributos que contienen valores nulos?, ¿alguno de dichos atributos son críticos para el análisis?

In [32]:
print("ℹ️ Valores nulos por columna:")
print(df.isnull().sum())

ℹ️ Valores nulos por columna:
Usuario X             0
ID                    0
Text                  0
Language              0
Type                  0
Author Name           0
Author Username       0
View Count            0
Bookmark Count        0
Favorite Count        0
Retweet Count         0
Reply Count           0
Quote Count           0
Created At            0
Source                0
Hashtags           3103
URLs               2774
Media Type         1938
Media URLs         1938
dtype: int64


👍 No se detectan atributos claves para el análisis que contengan valores nulos

🗒️ ¿Cuáles son los tipos de dato de cada columna?

In [33]:
print(f"Tipo de Dato por Columna:")
for columna, tipo in df.dtypes.items():
    print(f"🔍 {columna}: {tipo}")

Tipo de Dato por Columna:
🔍 Usuario X: object
🔍 ID: int64
🔍 Text: object
🔍 Language: object
🔍 Type: object
🔍 Author Name: object
🔍 Author Username: object
🔍 View Count: int64
🔍 Bookmark Count: int64
🔍 Favorite Count: int64
🔍 Retweet Count: int64
🔍 Reply Count: int64
🔍 Quote Count: int64
🔍 Created At: object
🔍 Source: object
🔍 Hashtags: object
🔍 URLs: object
🔍 Media Type: object
🔍 Media URLs: object


⚠️ Se observa que el atributo “Created At” fue almacenado en el dataframe como de tipo “object” cuando en realidad es de tipo datetime. Se procede con la corrección de tipo.

In [34]:
# Realizamos la corrección del tipo de dato para la columna “Created At”
df['Created At'] = pd.to_datetime(df['Created At'])

# Verificamos el nuevo tipo de dato para la columna “Created At”
print(f"🔍 El nuevo tipo de dato es: {df['Created At'].dtype}")

🔍 El nuevo tipo de dato es: datetime64[ns]


In [35]:
print(f"🔍 Resumen de los tipos de datos que componen el dataframe:")
resumen_tipos_datos = df.dtypes.value_counts()
print(resumen_tipos_datos)

🔍 Resumen de los tipos de datos que componen el dataframe:
object            11
int64              7
datetime64[ns]     1
Name: count, dtype: int64


🗒️ Veamos una pequeña muestra del dataframe, exploramos las primeras y últimas filas.

In [36]:
print("Esta es una muestra de los datos contenidos en el dataframe:")
print("🗂️ PRIMEROS 10 REGISTROS")
print("=========================")
df.head(10)

Esta es una muestra de los datos contenidos en el dataframe:
🗂️ PRIMEROS 10 REGISTROS


,Usuario X,ID,Text,Language,Type,Author Name,Author Username,View Count,Bookmark Count,Favorite Count,Retweet Count,Reply Count,Quote Count,Created At,Source,Hashtags,URLs,Media Type,Media URLs
1,JMilei,1888684208325075281,VIVA LA LIBERTAD CARAJO https://t.co/7As233e2PO,es,Tweet,Javier Milei,JMilei,47514,2,623,94,189,3,2025-02-09 17:19:39,Twitter for Android,NaN,https://www.instagram.com/reel/DF3X1fDJBBZ/?ig...,NaN,NaN
2,JMilei,1888636220445511716,TREMENDA MASTERCLAS del PROFE @jlespert al bru...,es,Tweet,Javier Milei,JMilei,1648346,297,17418,2500,1278,193,2025-02-09 14:08:58,Twitter for Android,NaN,NaN,NaN,NaN
3,JMilei,1888632601813885362,Creí que a mis 63 años ya lo había visto todo ...,es,Quoted,José Luis Espert,jlespert,1217423,451,13667,2269,847,137,2025-02-09 13:54:36,Twitter Web App,NaN,NaN,NaN,NaN
4,JMilei,1888440655422136478,NO APTO PARA SOCIALISTAS,es,Tweet,Javier Milei,JMilei,632983,1291,18885,4333,1142,139,2025-02-09 01:11:52,Twitter for Android,NaN,NaN,NaN,NaN
5,JMilei,1888340973530554371,¿Por qué el libertarismo no tiene nada que ver...,es,Quoted,Fundación Faro Argentina,fundfaro,951653,1568,8537,3154,383,233,2025-02-08 18:35:46,Twitter Web App,NaN,NaN,video,https://video.twimg.com/amplify_video/18883361...
6,JMilei,1888428020718862373,FENÓMENO BARRIAL,es,Tweet,Javier Milei,JMilei,406762,167,13241,1835,535,50,2025-02-09 00:21:40,Twitter for Android,NaN,NaN,NaN,NaN
7,JMilei,1888412358025838681,"ELON: I'M A BIG FAN OF MILEI ""He's doing fanta...",en,Quoted,Mario Nawfal,MarioNawfal,987028,188,4671,765,152,63,2025-02-08 23:19:25,Twitter Web App,NaN,NaN,video,https://video.twimg.com/ext_tw_video/188841227...
8,JMilei,1888389829567598645,VIVA LA LIBERTAD CARAJO https://t.co/nJzjio1dOU,es,Tweet,Javier Milei,JMilei,38314,2,499,70,82,6,2025-02-08 21:49:54,Twitter for Android,NaN,https://www.instagram.com/p/DF1SDIaJdsl/?igsh=...,NaN,NaN
9,JMilei,1888329760608002243,FENÓMENO BARRIAL,es,Tweet,Javier Milei,JMilei,223230,24,6192,661,435,43,2025-02-08 17:51:13,Twitter for Android,NaN,NaN,NaN,NaN
10,JMilei,1888320882088518008,Essential for general prosperity,ca,Quoted,Elon Musk,elonmusk,15328834,1640,99078,11470,2045,285,2025-02-08 17:15:56,Twitter for iPhone,NaN,NaN,NaN,NaN


In [37]:
print("🗂️ ÚLTIMOS 10 REGISTROS")
print("========================")
df.tail(10)

🗂️ ÚLTIMOS 10 REGISTROS


,Usuario X,ID,Text,Language,Type,Author Name,Author Username,View Count,Bookmark Count,Favorite Count,Retweet Count,Reply Count,Quote Count,Created At,Source,Hashtags,URLs,Media Type,Media URLs
3307,JMilei,1941571022471430363,FENÓMENO MUNDIAL https://t.co/cc5SJcEndu,es,Retweet,Ave Miller,gorrasdeljavo,39217,53,202,6,1356,16,2025-07-05 15:52:59,TweetDeck Web App,NaN,NaN,photo,https://pbs.twimg.com/media/GvHYgKgWsAAeeuC.jpg
3308,JMilei,1941533892848283918,Javier Milei dará un discurso en la inauguraci...,es,Retweet,Marina,Marinabiagettii,47770,46,44,3,278,0,2025-07-05 13:25:26,Twitter for Android,NaN,NaN,photo,https://pbs.twimg.com/media/GvG2yUcXUAAkusP.jpg
3309,JMilei,1941568307213255061,AURA https://t.co/coWTQo2nwc,eu,Retweet,SheIby,TommyShelby_30,27647,17,112,4,903,7,2025-07-05 15:42:11,Twitter for iPhone,NaN,NaN,photo,https://pbs.twimg.com/media/GvHWFTkXsAAqjNl.jpg
3310,JMilei,1941555643237994589,El Presidente Javier Milei junto al Primer Min...,es,Retweet,Oficina del Presidente,OPRArgentina,196324,101,429,17,3485,30,2025-07-05 14:51:52,Twitter for Android,NaN,NaN,photo,https://pbs.twimg.com/media/GvHKkRJXsAA3MCG.jpg
3311,JMilei,1941558663933722848,"FOTAAAAZAAAAA!!! 🇮🇳🇦🇷 ""Coincidimos en que el c...",es,Retweet,Marina,Marinabiagettii,34258,29,107,1,761,8,2025-07-05 15:03:52,Twitter for Android,NaN,NaN,photo,https://pbs.twimg.com/media/GvHNUJMWMAAFd85.jpg
3312,JMilei,1941520940799754296,🇦🇷🇮🇳 El presidente Javier Milei recibió en Ca...,es,Retweet,La Derecha Diario,laderechadiario,47473,62,181,23,1080,20,2025-07-05 12:33:58,Twitter for Android,NaN,NaN,video,https://video.twimg.com/amplify_video/19415001...
3313,JMilei,1941525477694505458,🇦🇷🤝🇮🇳Se viene tremendo acuerdo con la India! @...,es,Retweet,Agarra la Pala,agarra_pala,52514,88,203,10,1961,9,2025-07-05 12:52:00,Twitter Web App,NaN,NaN,photo,https://pbs.twimg.com/media/GvGuYQfW4AALF3s.jpg
3314,JMilei,1941332000205582752,JAVIER CUMPLE ✅️ 🇦🇷🔥 Qué lindo ver como se cum...,es,Retweet,Teresa Christina C.🦁💜🇦🇷,Tcristina77,28680,12,105,1,579,18,2025-07-05 00:03:11,Twitter for Android,NaN,NaN,video,https://video.twimg.com/ext_tw_video/194133197...
3315,JMilei,1941524058048430524,El abrazo de Javier Milei y el Primer Ministro...,es,Retweet,Marina,Marinabiagettii,28364,11,65,1,504,3,2025-07-05 12:46:21,Twitter for Android,NaN,NaN,photo,https://pbs.twimg.com/media/GvGt150XMAAfFws.jpg
3316,JMilei,1941555939162865911,"Y mientras tanto, en la mayoría de los medios ...",es,Retweet,𝙏𝙧𝙤𝙣𝙘𝙤,tronco,64183,111,339,3,2098,7,2025-07-05 14:53:02,Twitter for iPhone,NaN,NaN,NaN,NaN


Selección de tweets:<br>
- Dirigentes seleccionados (Author Username = ): JMilei <br>
- Type = Tweet <br>
- Fecha desde/hasta: completo <br>

In [38]:
condicion_username = df['Author Username'] == 'JMilei'
condicion_type = df['Type'] == 'Tweet'

# Creo un nuevo dataframe con la data filtrada
df_sentimientos = df[condicion_username & condicion_type].copy()

🗒️ Veamos una pequeña muestra del nuevo dataframe, exploramos las primeras filas.

In [39]:
print("Esta es una muestra de los datos contenidos en el dataframe:")
print("🗂️ PRIMEROS 10 REGISTROS")
print("=========================")
df_sentimientos.head(10)

Esta es una muestra de los datos contenidos en el dataframe:
🗂️ PRIMEROS 10 REGISTROS


,Usuario X,ID,Text,Language,Type,Author Name,Author Username,View Count,Bookmark Count,Favorite Count,Retweet Count,Reply Count,Quote Count,Created At,Source,Hashtags,URLs,Media Type,Media URLs
1,JMilei,1888684208325075281,VIVA LA LIBERTAD CARAJO https://t.co/7As233e2PO,es,Tweet,Javier Milei,JMilei,47514,2,623,94,189,3,2025-02-09 17:19:39,Twitter for Android,NaN,https://www.instagram.com/reel/DF3X1fDJBBZ/?ig...,NaN,NaN
2,JMilei,1888636220445511716,TREMENDA MASTERCLAS del PROFE @jlespert al bru...,es,Tweet,Javier Milei,JMilei,1648346,297,17418,2500,1278,193,2025-02-09 14:08:58,Twitter for Android,NaN,NaN,NaN,NaN
4,JMilei,1888440655422136478,NO APTO PARA SOCIALISTAS,es,Tweet,Javier Milei,JMilei,632983,1291,18885,4333,1142,139,2025-02-09 01:11:52,Twitter for Android,NaN,NaN,NaN,NaN
6,JMilei,1888428020718862373,FENÓMENO BARRIAL,es,Tweet,Javier Milei,JMilei,406762,167,13241,1835,535,50,2025-02-09 00:21:40,Twitter for Android,NaN,NaN,NaN,NaN
8,JMilei,1888389829567598645,VIVA LA LIBERTAD CARAJO https://t.co/nJzjio1dOU,es,Tweet,Javier Milei,JMilei,38314,2,499,70,82,6,2025-02-08 21:49:54,Twitter for Android,NaN,https://www.instagram.com/p/DF1SDIaJdsl/?igsh=...,NaN,NaN
9,JMilei,1888329760608002243,FENÓMENO BARRIAL,es,Tweet,Javier Milei,JMilei,223230,24,6192,661,435,43,2025-02-08 17:51:13,Twitter for Android,NaN,NaN,NaN,NaN
11,JMilei,1888298020938330160,Acá podemos observar cómo la basura de Brancat...,es,Tweet,Javier Milei,JMilei,4043267,2325,86470,13471,4399,721,2025-02-08 15:45:05,Twitter for Android,NaN,NaN,NaN,NaN
13,JMilei,1888287507558592575,BRILLANTE,es,Tweet,Javier Milei,JMilei,224010,27,5044,671,387,15,2025-02-08 15:03:19,Twitter for Android,NaN,NaN,NaN,NaN
15,JMilei,1887999293224542433,https://t.co/znZUzFRxn0,zxx,Tweet,Javier Milei,JMilei,89044,30,4832,754,497,47,2025-02-07 19:58:03,Twitter for Android,NaN,https://www.infobae.com/politica/2025/02/07/ja...,NaN,NaN
16,JMilei,1887948597679055312,AQUÍ MI OPINIÓN SOBRE EL TIPO DE CAMBIO Y LOS ...,es,Tweet,Javier Milei,JMilei,1785801,725,12808,2570,1184,252,2025-02-07 16:36:36,Twitter for Android,NaN,https://www.lanacion.com.ar/opinion/opinion-at...,NaN,NaN


##### ![6) Data Wrangling / Data Munging:](https://github.com/chetincho/ds_Prediccion_de_enfermedades/blob/main/img/Data%20Wrangling%20Munging.jpg?raw=true)

A continuacion vamos a proceder con los siguientes pasos claves para la preparación de los datos: <br>
<br>
`1. Limpieza del Texto:` Vamos a eliminar el posible ruido que contenga el atributo Text, campo que almacena el texto puro del twit publicado por el referente político. En este caso la limpieza busca eliminar: enlaces, menciones, hashtags, caracteres especiales, emojis, etc.<br>
🗒️Argumento: Limpiar el texto es crucial para que el modelo de análisis de sentimientos pueda realizar una correcta interpretación.<br>

Esto incluye tareas como:
- Eliminar URLs
- Eliminar menciones (@usuario)
- Eliminar hashtags (#hashtag)
- Eliminar signos de puntuación y caracteres especiales (a menos que sean relevantes para el sentimiento, como !)
- Convertir el texto a minúsculas
- Eliminar stopwords (palabras comunes que no aportan significado, como "el", "la", "un", etc.)
- Lematización (reducir palabras a su raíz, por ejemplo, "corriendo" a "correr").
- Vectorización

Tarea 1: Eliminar URL's<br>
Los tweets a menudo contienen enlaces a sitios web o imágenes los cuales no aportan valor al análisis de sentimiento y pueden incluso confundir el modelo.

In [40]:
def eliminar_urls(text):
    # Utilizamos expresiones regulares para encontrar URLs
    # Buscamos los siguientes patrones como http://, https://, www., o cualquier terminación de dominio (.com, .org, .net, etc.)
    url_pattern = re.compile(r'https?://\S+|www\.\S+|\S+\.(com|org|net|gov|edu|info|io|co|ar|es)[/\w.-]*')
    return url_pattern.sub(r'', text)

# Aplicamos la función a la columna 'Text' y creamos una nueva columna llamada "Texto_Limpio"
df_sentimientos['Texto_Limpio'] = df_sentimientos['Text'].apply(eliminar_urls)

🗒️ Mostramos los primeros 20 registros para confirmar la correcta limpieza de los datos.

In [41]:
print(df_sentimientos[['Text', 'Texto_Limpio']].head(20))

                                                 Text  \
1     VIVA LA LIBERTAD CARAJO https://t.co/7As233e2PO   
2   TREMENDA MASTERCLAS del PROFE @jlespert al bru...   
4                            NO APTO PARA SOCIALISTAS   
6                                    FENÓMENO BARRIAL   
8     VIVA LA LIBERTAD CARAJO https://t.co/nJzjio1dOU   
9                                    FENÓMENO BARRIAL   
11  Acá podemos observar cómo la basura de Brancat...   
13                                          BRILLANTE   
15                            https://t.co/znZUzFRxn0   
16  AQUÍ MI OPINIÓN SOBRE EL TIPO DE CAMBIO Y LOS ...   
17    VIVA LA LIBERTAD CARAJO https://t.co/eIMvBa2nmc   
18  VIVA LA LIBERTAD CARAJO CC: @Nikgaturro https:...   
19  FENÓMENO BARRIAL Este comentario le caerá muy ...   
20  FENÓMENO BARRIAL VIVA LA LIBERTAD CARAJO https...   
21    VIVA LA LIBERTAD CARAJO https://t.co/RUXTtO7OZd   
22                                         IMPERDIBLE   
24    VIVA LA LIBERTAD CARAJO h

Tarea 2: Eliminar menciones<br>
Estas menciones son importantes para la interacción en la plataforma pero generalmente no contribuyen al sentimiento general del tweet y pueden contribuir a generar ruido durante el análisis.

In [42]:
def eliminar_menciones(text):
    # Utilizamos expresiones regulares para encontrar las menciones
    # En este caso la expresión regular cominezan con '@' seguido de palabras o números
    menciones_pattern = re.compile(r'@\w+')
    return menciones_pattern.sub(r'', text)

# Aplicamos la función a la columna 'Texto_Limpio' pisando el contenido anterior
df_sentimientos['Texto_Limpio'] = df_sentimientos['Texto_Limpio'].apply(eliminar_menciones)

🗒️ Mostramos los primeros 20 registros para confirmar la correcta limpieza de los datos.

In [43]:
print(df_sentimientos[['Text', 'Texto_Limpio']].head(20))

                                                 Text  \
1     VIVA LA LIBERTAD CARAJO https://t.co/7As233e2PO   
2   TREMENDA MASTERCLAS del PROFE @jlespert al bru...   
4                            NO APTO PARA SOCIALISTAS   
6                                    FENÓMENO BARRIAL   
8     VIVA LA LIBERTAD CARAJO https://t.co/nJzjio1dOU   
9                                    FENÓMENO BARRIAL   
11  Acá podemos observar cómo la basura de Brancat...   
13                                          BRILLANTE   
15                            https://t.co/znZUzFRxn0   
16  AQUÍ MI OPINIÓN SOBRE EL TIPO DE CAMBIO Y LOS ...   
17    VIVA LA LIBERTAD CARAJO https://t.co/eIMvBa2nmc   
18  VIVA LA LIBERTAD CARAJO CC: @Nikgaturro https:...   
19  FENÓMENO BARRIAL Este comentario le caerá muy ...   
20  FENÓMENO BARRIAL VIVA LA LIBERTAD CARAJO https...   
21    VIVA LA LIBERTAD CARAJO https://t.co/RUXTtO7OZd   
22                                         IMPERDIBLE   
24    VIVA LA LIBERTAD CARAJO h

Tarea 3: Eliminar hashtags<br>
Los hashtags son elementos estructurales de Twitter que suelen agrupar temas, pero su presencia en el texto puede no ser útil para un análisis de sentimiento.

In [44]:
def eliminar_hashtags(text):
    # Utilizamos expresiones regulares para encontrar los hashtags
    # En este caso la expresión regular cominezan con '#' seguido de palabras o números
    hashtag_pattern = re.compile(r'#\w+')
    return hashtag_pattern.sub(r'', text)

# Aplicamos la función a la columna 'Texto_Limpio' pisando el contenido anterior
df_sentimientos['Texto_Limpio'] = df_sentimientos['Texto_Limpio'].apply(eliminar_hashtags)

🗒️ Mostramos una selección de filas donde sabemos que hay un hashtag.

In [45]:
# Ajustar el seteo de pandas para que se muestre el contenido de toda la fila y no lo trunque
pd.set_option('display.max_colwidth', None)

# mostramos un extracto donde sabemos que hay un #
print(df_sentimientos[['Text', 'Texto_Limpio']].loc[610:650])

# Devolvemos este seteo a los valores por defecto.
pd.set_option('display.max_colwidth', 50)

                                                                                                                                                                                                                                                                                      Text  \
611                                                                                                                                                                                                                                                                       FENÓMENO BARRIAL   
613                                                                                                                                                                                                                                                                       FENÓMENO BARRIAL   
615                                                                                                                                           

Tarea 4: Eliminar signos de puntuación y caracteres especiales<br>
Al tratearse de un análisis de texto en español es crucial conservar las ñ y las tildes (á, é, í, ó, ú, ü) junto con el símbolo de exclamación (!)

In [46]:
def eliminar_caracteres_especiales(text):
    # 'a-zA-Z' cubre las letras básicas.
    # '0-9' cubre los números.
    # '\s' cubre los espacios en blanco.
    # '!' conserva el signo de exclamación.
    # 'ñÑáÁéÉíÍóÓúÚüÜ' cubren los caracteres específicos del español.
    texto_limpio = re.sub(r'[^a-zA-Z0-9\s!ñÑáÁéÉíÍóÓúÚüÜ]', '', text)
    return texto_limpio

# Aplicamos la función a la columna 'Texto_Limpio' pisando el contenido anterior
df_sentimientos['Texto_Limpio'] = df_sentimientos['Texto_Limpio'].apply(eliminar_caracteres_especiales)

🗒️ Mostramos una selección de filas donde sabemos que hay caracteres especiales.

In [47]:
# Ajustar el seteo de pandas para que se muestre el contenido de toda la fila y no lo trunque
pd.set_option('display.max_colwidth', None)

# mostramos un extracto donde sabemos que hay un #
print(df_sentimientos[['Text', 'Texto_Limpio']].loc[610:650])

# Devolvemos este seteo a los valores por defecto.
pd.set_option('display.max_colwidth', 50)

                                                                                                                                                                                                                                                                                      Text  \
611                                                                                                                                                                                                                                                                       FENÓMENO BARRIAL   
613                                                                                                                                                                                                                                                                       FENÓMENO BARRIAL   
615                                                                                                                                           

Paso 5: Convertir el texto a minúsculas<br>
Motivos por los cuales la conversión a minúsculas es fundamental en cualquier Procesamiento de Lenguaje Natural (PLN).<br>
<br>

- `Normalización:` Trata de igual manera palabras con diferente capitalización (ej. "Libertad", "libertad", "LIBERTAD"). Esto es esencial para que un modelo de análisis de sentimientos no las vea como entidades distintas.

- `Reducción de Vocabulario:` Disminuye el tamaño del vocabulario, lo que conlleva una mejora en el rendimiento y la eficiencia de los modelos. Por ejemplo, en lugar de tener "Libertad", "LIBERTAD" y "libertaD", solo tendremos "libertad".

- `Consistencia:` Asegura que las comparaciones y el conteo de palabras sean consistentes en todo el dataset.

In [48]:
def convertir_a_minusculas(text):
    return text.lower()

# Aplicamos la función a la columna 'Texto_Limpio' pisando el contenido anterior
df_sentimientos['Texto_Limpio'] = df_sentimientos['Texto_Limpio'].apply(convertir_a_minusculas)

🗒️ Mostramos una selección de filas donde sabemos que hay mayúsculas.

In [49]:
# Ajustar el seteo de pandas para que se muestre el contenido de toda la fila y no lo trunque
pd.set_option('display.max_colwidth', None)

# mostramos un extracto donde sabemos que hay un #
print(df_sentimientos[['Text', 'Texto_Limpio']].loc[610:650])

# Devolvemos este seteo a los valores por defecto.
pd.set_option('display.max_colwidth', 50)

                                                                                                                                                                                                                                                                                      Text  \
611                                                                                                                                                                                                                                                                       FENÓMENO BARRIAL   
613                                                                                                                                                                                                                                                                       FENÓMENO BARRIAL   
615                                                                                                                                           

Paso 6: Eliminar stopwords<br>
- Fundamentos:<br>
La eliminación de stopwords es un paso crítico en el preprocesamiento de texto referido a tareas de análisis de sentimientos. Las stopwords son palabras muy comunes en un idioma que, por lo general, no aportan un significado semántico relevante para el análisis. Entre las stopwords encontramos: `artículos` (el, la, un), `preposiciones` (de, en, para), `conjunciones` (y, o), `pronombres` (yo, tú), y `verbos auxiliares`, entre otras.<br>
<br>
- ¿Por qué es importante este paso?<br>
    1. `Reducción de Ruido:` Estas palabras aparecen con altísima frecuencia en casi cualquier texto, pero no suelen llevar la carga semántica principal del mensaje. Por ejemplo, en la frase "La película es muy buena para ver el sábado", las palabras clave que indican el sentimiento son `"muy"` y `"buena"`. "La", "es", "para", "ver", "el", "sábado" son stopwords o palabras de bajo valor que simplemente estructuran la oración. Eliminarlas reduce el "ruido" que podría distraer al modelo.<br>
    <br>
    2. `Optimización del Rendimiento:` Al remover las stopwords, se reduce significativamente el número total de palabras en tu conjunto de datos. Esto tiene dos beneficios directos:<br>
    <br>
        - `Menor dimensionalidad:` Cuando transformas un texto en representaciones numéricas (como vectores de palabras), tener menos palabras únicas reduce la cantidad de dimensiones, lo que puede acelerar el entrenamiento de los modelos y hacerlos más eficientes computacionalmente.<br>
        - `Menor uso de memoria:` Un vocabulario más pequeño requiere menos memoria tanto en el almacenamiento como durante el procesamiento.<br>
    <br>
    3. `Mejora de la Precisión del Modelo:` Al centrar el análisis en las palabras que realmente transmiten significado, los modelos de análisis de sentimientos pueden identificar patrones más claros y relevantes. Si las stopwords permanecen, podrían ser falsamente consideradas como características importantes debido a su alta frecuencia, diluyendo el peso de las palabras que sí son significativas para el sentimiento. En un análisis de sentimiento, queremos que el modelo aprenda de palabras como "increíble", "terrible", "feliz", "triste", y no de "el", "a", "de".

In [50]:
# Programación defensiva #
# Con este bloque try-except nos aseguramos que si aun no se descargaron las stopwords se descarguen automaticamente la primera vez que ejecutemos el script.
# ---------------------- #

try:
    # Carga en la variable stopwords_ES la lista de stopwords para el idioma español.
    # Esta variable es de tipo set (conjunto).
    stopwords_ES = stopwords.words('spanish')
except LookupError:
    # En caso de error procede con la descarga de las listas de stopwords
    nltk.download('stopwords')
    stopwords_ES = stopwords.words('spanish')

In [51]:
def quitar_stopwords(text):
    
    # Paso 1: Tokenizamos el texto para separar las palabras
    palabras = text.split()
    
    # Paso 2: Filtramos las palabras que no están en nuestra lista de stopwords
    palabras_filtradas = [palabra 
                                for palabra in palabras
                                    if palabra not in stopwords_ES
                         ]
    # Unimos las palabras en una cadena
    return ' '.join(palabras_filtradas)


# Aplicamos la función a la columna 'Texto_Limpio' pisando el contenido anterior
df_sentimientos['Texto_Limpio'] = df_sentimientos['Texto_Limpio'].apply(quitar_stopwords)

🗒️ Mostramos una selección de filas donde sabemos que hay stopwords.

In [52]:
# Ajustar el seteo de pandas para que se muestre el contenido de toda la fila y no lo trunque
pd.set_option('display.max_colwidth', None)

# mostramos un extracto donde sabemos que hay un #
print(df_sentimientos[['Text', 'Texto_Limpio']].loc[610:650])

# Devolvemos este seteo a los valores por defecto.
pd.set_option('display.max_colwidth', 50)

                                                                                                                                                                                                                                                                                      Text  \
611                                                                                                                                                                                                                                                                       FENÓMENO BARRIAL   
613                                                                                                                                                                                                                                                                       FENÓMENO BARRIAL   
615                                                                                                                                           

- Texto previo a la remoción de `stopwords`<br>
644  .@rolandogps operador inmundo parece `que` `no` `has` entendido `las` condiciones. Tenés `que` hacerlo `en` `tu` programa pidiendo disculpas `y` admitir `que` `has` mentido `de` modo intencionado. `Al` mismo tiempo vos `y` `tu` grupo deberán donar $ 5M . CIAO! CC: @FOPEA háganse cargo `porque` uds. apañan.
<br>
<br>
- Texto final luego de la remocion de `stopwords`<br>
644  operador inmundo parece entendido condiciones tenés hacerlo programa pidiendo disculpas admitir mentido modo intencionado mismo tiempo vos grupo deberán donar 5m ciao! cc háganse cargo uds apañan

Paso 7: Lematización<br>
<br>
- `Qué es:` Es el proceso por el cual se reduce las palabras a su forma base canónica o lema, es decir, la forma en la cual encontraríamos la palabra en un diccionario. Esta técnica demanda conocer el idioma desde el punto de vista morfológico y parte del habla de la palabra.<br>
<br>
- `Cómo funciona:` Esta técnica utiliza diccionarios y análisis morfológico para transformar la palabra a su lema correcto. Por ejemplo, reconoce que "corriendo", "corrí", "correré" provienen del verbo "correr".<br>
<br>
- `Ventajas:` Produce palabras reales y gramaticalmente correctas, lo que puede ser beneficioso para la interpretabilidad y para modelos que dependen de significados léxicos.<br>
<br>
- `Desventajas:` Es un proceso lento y computacionalmente costoso, ya que requiere más recursos (diccionarios, reglas gramaticales).<br>
<br>
- `Ejemplos:`<br>
        - corriendo -> correr<br>
        - correr -> correr<br>
        - corro -> correr<br>
        - belleza -> belleza<br>
        - universal -> universal<br>
        - universidad -> universidad<br>
<br>
- `¿Por qué este paso es importante?`<br>
La lematización es fundamental en PLN por las siguientes razones:<br>
<br>
    1. `Normalización Adicional:` Estamos dando un paso mas, no nos limitamos únicamente a la conversión a minúsculas y la eliminación de stopwords. Consolidan diferentes formas flexivas de una misma palabra en una única forma base.<br>
    2. `Mejora la Precisión del Modelo:` Al agrupar estas variantes, el modelo de análisis de sentimientos puede reconocer que todas se refieren al mismo concepto. Esto evita que el modelo trate "bueno", "buenas", "buena", "buenos" como cuatro palabras distintas que deben aprender a clasificar, y en su lugar, aprende del lema "bueno", concentrando la "fuerza" de todas esas apariciones en un solo token.<br>
    3. `Reducción de Dimensionalidad del Vocabulario:` Esto se asimila a la eliminación de stopwords, pero a un nivel más granular, este paso reduce aún más el número total de palabras únicas en el corpus. Un vocabulario más compacto significa menos características para los modelos de aprendizaje automático, lo que puede conducir a:<br>
        - Entrenamiento más rápido: Menos parámetros para ajustar.<br>
        - Menor riesgo de sobreajuste (overfitting): Los modelos se vuelven más generalizables al centrarse en los conceptos centrales y no en las inflexiones gramaticales.<br>
        - Menor demanda de memoria.
<br>
- Implementación de SpaCy: En la terminal correr estos comandos:<br>
    - `pip install spacy`  : Instala la librería spaCy.<br>
    - `python -m spacy download es_core_news_sm`  : Descarga el modelo de lenguaje español.<br>

In [53]:
# Programación defensiva #
# Cargamos el modelo de lenguaje español de spaCy, esto se hace solo una vez.
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    print("El modelo 'es_core_news_sm' no está instalado. Ejecutar en la terminal: python -m spacy download es_core_news_sm")
    exit()

In [ ]:
def lematizacion(text):
    if pd.isna(text): return text
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc if not token.is_space]
    return ' '.join(lemmas)

# Aplicamos la función a la columna 'Texto_Limpio' pisando el contenido anterior
df_sentimientos['Texto_Limpio'] = df_sentimientos['Texto_Limpio'].apply(lematizacion)

🗒️ Mostramos una selección de filas para verificar la lematización.

In [55]:
# Ajustar el seteo de pandas para que se muestre el contenido de toda la fila y no lo trunque
pd.set_option('display.max_colwidth', None)

# mostramos un extracto donde sabemos que hay un #
print(df_sentimientos[['Text', 'Texto_Limpio']].loc[610:650])

# Devolvemos este seteo a los valores por defecto.
pd.set_option('display.max_colwidth', 50)

                                                                                                                                                                                                                                                                                      Text  \
611                                                                                                                                                                                                                                                                       FENÓMENO BARRIAL   
613                                                                                                                                                                                                                                                                       FENÓMENO BARRIAL   
615                                                                                                                                           

- Texto previo a la `lematizacion`<br>
638  FENÓMENO BARRIAL `MANDRILES` `traten` de superar su estrepitoso fracaso porque en su defecto `van` a vivir `llorando`...
<br>
<br>
- Texto final luego de la `lematizacion`<br>
638  fenómeno barrial mandril tratar superar estrepitoso fracaso defecto ir vivir llorar

------------------------------------------

`2. Selección de una Herramienta/Modelo:` Existen diversas librerías y enfoques para el análisis de sentimientos en Python. Algunos son más simples (basados en léxicos), otros más complejos (basados en modelos de Machine Learning/Deep Learning).

`3. Aplicación del Análisis:` Ejecutar la herramienta seleccionada sobre la columna de texto de tus tweets.

`4. Interpretación y Visualización:` Entender los resultados, cuántos tweets son positivos, negativos o neutros, y quizás visualizar estas distribuciones.

##### ![Primeras conclusiones:](https://github.com/chetincho/ds_Prediccion_de_enfermedades/blob/main/img/Primeras%20conclusiones.jpg?raw=true)

<br>
<br>
<br>
<br>
<br>
<br>


##### ![Selección y entrenamiento del modelo:](https://github.com/chetincho/ds_Prediccion_de_enfermedades/blob/main/img/Selecci%C3%B3n%20y%20entrenamiento%20del%20modelo%20fundamentos.jpg?raw=true)

<br>
<br>
<br>
<br>
<br>
<br>


##### ![Conclusiones finales:](https://github.com/chetincho/ds_Prediccion_de_enfermedades/blob/main/img/Conclusiones%20finales.jpg?raw=true)

<br>
<br>
<br>
<br>
<br>
<br>